In [4]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for

In [6]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

def request_chatgpt_gpt4(messages):
    original_url = "http://127.0.0.1:5000/citation_event_hgraph"
    body = {"messages": messages}
    response = requests.post(original_url, json=body).json()
    gpt_response = response['choices'][0]['message']['content'].strip()
    return gpt_response

In [46]:
def citation_messages(content, example_content=None, example_summary=None):
    if example_content is None:
        messages = [
            { 
            "role": "system", 
            # "content": """
            #     You are a system that helps researchers understand how a citation is used in a paper.
            #     The user will give you a paragraph that contains a special token: [[target]].
            #     There will be other citations in the paragraph, marked by a number with square brackets, such as [1].
            #     Focus only on the target citation and only mention others when necessary.
            #     You will give the user a concise explanation on why the citation is used in the paragraph.
            # """
            "content": """
                You are a system that helps researchers understand the relations between citations.
                The user will give you a paragraph that contains one or more citations marked by a number with square brackets, such as [1].
                Keep in mind that citations with the same number are the same citation.
                First, group the citations into clusters based on their relations, so that citations in the same cluster are highly-correlated to each other.
                Then for each group, assign a noun-phrase that best describes the citations within.
            """
            },
            # { "role": "system", "name": "example_user", "content": example_content},
            # { "role": "system", "name": "example_system", "content": example_summary},
            { "role": "user", "content": content}
        ]
    else:
        messages = [
            { 
            "role": "system", 
            "content": """
                You are a system that helps researchers understand how a citation is used in a paper.
                The user will give you a paragraph that contains a special token: [[target]].
                There will be other citations in the paragraph, marked by a number with square brackets, such as [1].
                Focus only on the target citation and only mention others when necessary.
                You will give the user a concise explanation on why the citation is used in the paragraph.
            """
            },
            { "role": "system", "name": "example_user", "content": example_content},
            { "role": "system", "name": "example_system", "content": example_summary},
            { "role": "user", "content": content}
        ]
    citation_events = request_chatgpt_gpt4(messages)
    return citation_events

In [39]:
content = """
Real-time monitoring of incoming orders. 
Warehouse managers can be aware of delayed orders through intuitive monitoring. 
However, the warehouse faces rapidly incoming orders from online users in real time, which brings an uncertain workload to monitoring the status of order processing. 
Moreover, the streaming data of order processing in e-commerce warehouses have complex features. 
First, orders have hierarchy data structures. 
For example, one order can include multiple stock keeping units (SKUs) to pick and a few packages to pack. 
Second, different orders are processed simultaneously in parallel operation stations. 
These features bring challenges to the existing visual analytics approach [target] on monitoring assembly lines in smart factories, which has limitations in presenting parallel tasks and hierarchy data structures. 
Thus, we need a new visual design to monitor complex incoming order data in real-time.
"""
content = """
Following the nine-stage design study methodology proposed by Sedl- mair et al. [target], we collaborated closely with two warehousing experts from a large e-commerce corporation. Both of them have worked in e-commerce warehousing for more than four years. Specifically, the col- laboration started with characterizing domain problems for e-commerce warehouses in the Precondition Phase. We iteratively experienced the Learn and Winnow stages at this phase, in which we arranged multiple interviews with experts, conducted field inspections at collaborated warehouses, and surveyed related literature and commercial applica- tions. Initial requirements were derived to monitor order processing status and solve delayed orders. Then, in the Core Phase, we held weekly meetings for four months with each other to discover specific requirements and tasks, and design prototypes to collect experts’ feed- back for improvements. The final requirements and visual design were derived in the iterative user-centric procedure including interviewing, observing, brainstorming, designing, and prototyping. We formulate design requirements in three aspects, namely, monitoring, analyzing, and evaluating delayed orders.
"""
content = """ 
The previous studies about manufacturing process visualization [29, 68] are highly related to our work. 
However, these studies assume that the dealing workload and producing schedule are stable. 
The situa- tion in e-commerce warehousing is different because the individual at- tributes (such as order SKUs) of online orders are unpredictable, which brings an uncertain processing workload. 
These uncertain workloads further result in a dynamic producing schedule due to specific order- picking operational strategies [11]. 
All these uncertainties make it hard to directly judge delaying events based on the spent time interval [29] and the proposed time-aware outlier-preserving visual aggregation tech- nique [68]. 
Inspired by these studies, we propose new visual designs and develop a novel system for monitoring, analyzing, and evaluating order delaying issues in e-commerce warehouses.
"""



In [43]:
related_work_content = """
Several visualization methods were proposed to analyze streaming data for event detection [18, 31], situation awareness [21], and dimensionality reduction [22]. 
Huron et al. [28] introduced an innovative visual metaphor called visual sedimentation to visualize streaming data and smooth the incoming transition of continuously updating data. 
Other metaphors such as dial [20] and storyline [52] were also proposed for efficient visualizations. 
Krstajic et al. [30] developed CloudLines, a novel visualization technique that can present event episodes in online news streams. 
Researchers also developed novel visualization systems to analyze continuously updating information in text data [38, 63] and spatiotemporal data [14, 15].
"""
related_work_content = """
Process visualization is a sub-field of information visualization, which has been used to monitor instrument conditions and producing pro- cesses since the industrial age [40]. 
With the recent development of digital instruments and smart facilities in industrial factories, many visu- alization methods are proposed to visualize the manufacturing process data and support specific analyzing requirements of different domains. 
Matkovic ́ et al. [40] integrated levels of detail and foucs+context tech- nique for virtual instrument process monitoring. 
Aigner et al. [6] introduced a novel glyph design called PlanningLines that is extended from LifeLines [46] to represent temporal uncertainties in project man- agement and medical treatment planning.
Researchers have explored several types of visualization to present process data, for example, Path- way Waterfall [32], the Gantt chart [6, 29] and Marey’s graph [44, 68]. 
LiveGantt [29] is a novel schedule visualization tool that is integrated with a layout algorithm and interactions to improve the scalability of conventional Gantt charts. 
Marey’s graph is a classical visualization for illustrating public transportation schedules [55]. 
Inspired by Marey’s graph, Palomo et al. [44] introduced an innovative visual analytics system called TR-EX, which applied kernel density estimation for pre- senting a mass of transportation data at multiple scales. 
Moreover, based on Marey’s graph, Xu et al. [68] proposed ViDX, a novel visual analytics system to diagnose abnormal events of assembly lines in smart factories. 
They applied a time-aware, outlier-preserving visual aggre- gation technique on the graph to reveal the historical anomalies and designed a radial graph to monitor the real-time processes. 
However, the system lacked details of parallel working stations and faced scalabil- ity problems. 
Besides visualizing the temporal processes, researchers also proposed approaches for visualizing processes in monitoring facil- ity conditions [62, 72], simulating fluid [8], analyzing sports [66, 69], and exploring abnormal patterns [67, 70].
"""
example_summary = """
The target citation is a visualization method proposed for analyzing streaming data for event detection,
which is one of other visualization methods mentioned in this paragraph as related work.
"""

In [47]:
res = citation_messages(related_work_content)
# res = citation_messages(content, example_content, example_summary)
print(res)

Based on the paragraph, we can group the citations into the following clusters:

Cluster 1: [40], [40], [40]
- These citations refer to the same paper by Matkovic et al. that integrated levels of detail and foucs+context technique for virtual instrument process monitoring.

Cluster 2: [6], [46]
- These citations refer to Aigner et al.'s paper that introduced a novel glyph design called PlanningLines, which is extended from LifeLines, to represent temporal uncertainties in project management and medical treatment planning.

Cluster 3: [32], [6], [29], [44], [68], [29], [55], [44], [68]
- These citations refer to different papers that explore various types of visualization methods for presenting process data. The papers mentioned are about Pathway Waterfall, the Gantt chart, Marey's graph, LiveGantt, Palomo et al.'s TR-EX, and Xu et al.'s ViDX.

Cluster 4: [44], [68]
- These citations refer to Palomo et al.'s innovative visual analytics system called TR-EX and Xu et al.'s novel visual an